In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df = pd.read_csv('/content/drive/My Drive/wine_categorization/data/test.csv')
df.shape

(20665, 11)

In [0]:
df.isna().sum()

user_name              4738
country                   4
review_title              0
review_description        0
designation            5989
points                    0
price                  1394
province                  4
region_1               3314
region_2              11751
winery                    0
dtype: int64

In [0]:
df_1 = df.loc[df.points.notna(),['review_title', 'review_description', 'points']]
display(df_1.head)

<bound method NDFrame.head of                                             review_title  ... points
0      Boedecker Cellars 2011 Athena Pinot Noir (Will...  ...     88
1      Mendoza Vineyards 2012 Gran Reserva by Richard...  ...     90
2                    Prime 2013 Chardonnay (Coombsville)  ...     87
3      Bodega Cuarto Dominio 2012 Chento Vineyard Sel...  ...     91
4               SassodiSole 2012  Brunello di Montalcino  ...     90
...                                                  ...  ...    ...
20660  Yorkville Cellars 2013 Rennie Vineyard Caberne...  ...     91
20661  Château Ribaute 2015 Senhal d'Aric Rosé (Corbi...  ...     84
20662  Daou 2014 Reserve Cabernet Sauvignon (Paso Rob...  ...     94
20663  Peñascal 2011 Tempranillo Rosé (Vino de la Tie...  ...     80
20664  Langtry 2005 Tephra Ridge Vineyard Cabernet Sa...  ...     87

[20665 rows x 3 columns]>

In [0]:
with open('/content/drive/My Drive/wine_categorization/model/Wine_dataset_test.pickle', 'wb') as output:
    pickle.dump(df_1, output)

In [0]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [0]:
path_df = "/content/drive/My Drive/wine_categorization/model/Wine_dataset_test.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [0]:
df.loc[6]['review_description']

'A structured wine that has berry fruits, black currants and a firm, dry core of tannins. It has the weight, texture and richness of the vintage, with a solid future as it rounds out and softens. Drink from 2015.'

In [0]:
df['Description_Parsed_1'] = df['review_description'].str.replace('"', '')

In [0]:
df['Description_Parsed_2'] = df['Description_Parsed_1'].str.lower()

In [0]:
punctuation_signs = list("?:!.,;")
df['Description_Parsed_3'] = df['Description_Parsed_2']

for punct_sign in punctuation_signs:
    df['Description_Parsed_3'] = df['Description_Parsed_3'].str.replace(punct_sign, '')

In [0]:
df['Description_Parsed_4'] = df['Description_Parsed_3'].str.replace("'s", "")

In [0]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Description_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [0]:
df['Description_Parsed_5'] = lemmatized_text_list

In [0]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
stop_words[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [0]:
df['Description_Parsed_6'] = df['Description_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Description_Parsed_6'] = df['Description_Parsed_6'].str.replace(regex_stopword, '')

In [0]:
df.loc[5]['review_description']

'The timid nose squeaks out notes of Chinese five spice and warm green apple, with a slight hint of sulfur. Light to medium bodied, the palate bears a slight sweetness, tame acids and a quick finish.'

In [0]:
df.loc[5]['Description_Parsed_1']

'The timid nose squeaks out notes of Chinese five spice and warm green apple, with a slight hint of sulfur. Light to medium bodied, the palate bears a slight sweetness, tame acids and a quick finish.'

In [0]:
df.loc[5]['Description_Parsed_2']

'the timid nose squeaks out notes of chinese five spice and warm green apple, with a slight hint of sulfur. light to medium bodied, the palate bears a slight sweetness, tame acids and a quick finish.'

In [0]:
df.loc[5]['Description_Parsed_5']

'the timid nose squeak out note of chinese five spice and warm green apple with a slight hint of sulfur light to medium body the palate bear a slight sweetness tame acids and a quick finish'

In [0]:
df['Description_Parsed_7']=df['Description_Parsed_6'].str.replace("'","")

In [0]:
df.loc[5]['Description_Parsed_7']

' timid nose squeak  note  chinese five spice  warm green apple   slight hint  sulfur light  medium body  palate bear  slight sweetness tame acids   quick finish'

In [0]:
df.head(1)

,review_title,review_description,points,Description_Parsed_1,Description_Parsed_2,Description_Parsed_3,Description_Parsed_4,Description_Parsed_5,Description_Parsed_6,Description_Parsed_7
0,Boedecker Cellars 2011 Athena Pinot Noir (Will...,Nicely differentiated from the companion Stewa...,88,Nicely differentiated from the companion Stewa...,nicely differentiated from the companion stewa...,nicely differentiated from the companion stewa...,nicely differentiated from the companion stewa...,nicely differentiate from the companion stewar...,nicely differentiate companion stewart bottl...,nicely differentiate companion stewart bottl...


In [0]:
list_columns = ["review_title", "review_description", "Description_Parsed_7", "points"]
df = df[list_columns]

df = df.rename(columns={'Description_Parsed_7': 'Description_Parsed'})

In [0]:
df.head(1)

,review_title,review_description,Description_Parsed,points
0,Boedecker Cellars 2011 Athena Pinot Noir (Will...,Nicely differentiated from the companion Stewa...,nicely differentiate companion stewart bottl...,88


In [0]:
variety_codes = {
    'Bordeaux-style Red Blend':0,
    'Bordeaux-style White Blend':1,
    'Cabernet Franc': 2,
    'Cabernet Sauvignon':3,
    'Champagne Blend':4,
    'Chardonnay':5,
    'Gamay':6,
    'Gewürztraminer':7,
    'Grüner Veltliner':8,
    'Malbec':9,
    'Merlot':10,
    'Nebbiolo':11,
    'Pinot Grigio':12,
    'Pinot Gris':13,
    'Pinot Noir':14,
    'Portuguese Red':15,
    'Portuguese White':16,
    'Red Blend':17,
    'Rhône-style Red Blend':18,
    'Riesling':19,
    'Rosé':20,
    'Sangiovese':21,
    'Sauvignon Blanc':22,
    'Sparkling Blend':23,
    'Syrah':24,
    'Tempranillo':25,
    'White Blend':26,
    'Zinfandel':27
}

In [0]:
X_val = df['Description_Parsed']

In [0]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [0]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_val = tfidf.fit_transform(X_val).toarray()
print(features_val.shape)

(20665, 300)


In [0]:
%cd /content/drive/My Drive/wine_categorization/model/

/content/drive/My Drive/wine_categorization/model


In [0]:
# X_val
with open('Pickle/X_val.pickle', 'wb') as output:
    pickle.dump(X_val, output)
# features_val
with open('Pickle/features_val.pickle', 'wb') as output:
    pickle.dump(features_val, output)


In [0]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [0]:
path_features_val = "/content/drive/My Drive/wine_categorization/model/Pickle/features_val.pickle"
with open(path_features_val, 'rb') as data:
    features_val = pickle.load(data)

In [0]:
print(features_val.shape)

(20665, 300)
